## Quick Start Tutorial 

### Example 1: use NwmHs class to download data (Recommended method)

Import NwmHs class and download data with get_data() method. Details of parameter settings for get_data() method could be found [here](https://nwm.readthedocs.io/en/latest/#parameter-settings).

In [ ]:
import matplotlib.pyplot as plt
from nwm import NwmHs

# get data from National water model HydroShare App
dataset = NwmHs().get_data(archive='harvey', config='short_range', geom='channel_rt',
                           variable='streamflow', comid=[5781915], init_time=0, 
                           start_date='2017-08-23')

The downloaded NWM dataset is stored as a self-described xarray object. With this data object, you can check the metadata information and make time series plot.

In [ ]:
# show metadata
dataset.attrs

# plot data
plt.figure(figsize=(9,5))
dataset.plot()
plt.xlabel('Year 2017')
plt.ylabel('{} ({})'.format(dataset.variable_name,dataset.variable_unit))
plt.title('Short range streamflow forecast for Channel 5781915 during Harvey Hurricane Event')

### Example 2: use BmiNwmHs class to download data (Demonstration of how to use BMI)

Import BmiNwmHs class and instantiate it. A configuration file (yaml file) is required to provide the parameter settings for data download. An example configure_file.yaml file is provided in the same folder with this Jupyter Notebook file. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cftime

from nwm import BmiNwmHs

# initiate a data component
data_comp = BmiNwmHs()
data_comp.initialize('config_file.yaml')

Use variable related methods from BmiNwmHs class to check the variable information of the NWM dataset. This data component stores a flow forecast variable. 

In [ ]:
# get variable info
var_name = data_comp.get_output_var_names()[0]
var_unit = data_comp.get_var_units(var_name)
print(' variable_name: {}\n var_unit: {}\n'.format(var_name, var_unit))

Use time related methods of BmiNwmHs class to check the time information of the NWM dataset. The time values are stored in a format which follows [CF convention](http://cfconventions.org/Data/cf-conventions/cf-conventions-1.8/cf-conventions.pdf). 

In [ ]:
# get time info
start_time = data_comp.get_start_time()
end_time = data_comp.get_end_time()
time_step = data_comp.get_time_step()
time_unit = data_comp.get_time_units()
time_steps = int((end_time - start_time)/time_step) + 1
print(' start_time:{}\n end_time:{}\n time_step:{}\n time_unit:{}\n time_steps:{}\n'.format(start_time, end_time, time_step, time_unit, time_steps))

Loop through each time step to get the flow and time values. stream_array stores flow forecast values. cftime_array stores the numerical time values. time_array stores the corresopnding Python datetime objects. get_value() method returns the flow forecast value at each time step. update() method updates the current time step of the data component.

In [ ]:
# initiate numpy arrays to store data
stream_value = np.empty(1)
stream_array = np.empty(time_steps)
cftime_array = np.empty(time_steps)

for i in range(0, time_steps):
    data_comp.get_value(var_name, stream_value)
    stream_array[i] = stream_value
    cftime_array[i] = data_comp.get_current_time()
    data_comp.update()
    
time_array = cftime.num2date(cftime_array, time_unit, only_use_cftime_datetimes=False, only_use_python_datetimes=True)

In [ ]:
# plot data
plt.figure(figsize=(9,5))
plt.plot(time_array, stream_array)
plt.xlabel('Year 2017')
plt.ylabel('{} ({})'.format(var_name, var_unit))
plt.title('Short range streamflow forecast for Channel 5781915 during Harvey Hurricane Event')